In [8]:
import torch
from transformers import BertForQuestionAnswering 
from transformers import BertTokenizer 
import pandas as pd 
import numpy as np
import tkinter as tk

#model 
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#Tokenizer 
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

class App: 
    def __init__(self):
        self.HEIGHT = 700
        self.WIDTH = 800    
        root = tk.Tk()
        root.width = self.WIDTH
        root.height = self.HEIGHT
        self.dialogroot = root
        self.strDialogResult = ""    
        self.canvas = tk.Canvas(root, height=self.HEIGHT, width=self.WIDTH)
        self.canvas.pack()    
        frame = tk.Frame(root, bg='#42c2f4')
        frame.place(relx=0.5, rely=0.02, relwidth=0.96, relheight=0.95, anchor='n')  
        # Here is the button call to the InputBox() function
        buttonInputBox = tk.Button(frame, text="Input Box", bg='#cccccc', font=60, 
        command=lambda: self.InputBox())    
        buttonInputBox.place(relx=0.05, rely=0.1, relwidth=0.90, relheight=0.8)    
        root.mainloop()

    def InputBox(self): 
            dialog = tk.Toplevel(self.dialogroot)
            dialog.width = 600
            dialog.height = 100
            
            frame = tk.Frame(dialog, bg='#42c2f4', bd=5)
            frame.place(relwidth=1, relheight=1)

            entry = tk.Entry(frame, font=40)
            entry.place(relwidth=0.65, rely=0.02, relheight=0.96)
            entry.focus_set()

            submit = tk.Button(frame, text='OK', font=16, command=lambda: self.DialogResult(entry.get()))
            submit.place(relx=0.7, rely=0.02, relheight=0.96, relwidth=0.3)

            root_name = self.dialogroot.winfo_pathname(self.dialogroot.winfo_id())
            dialog_name = dialog.winfo_pathname(dialog.winfo_id())
            
            self.dialogroot.tk.eval('tk::PlaceWindow {0} widget {1}'.format(dialog_name, root_name))

            self.dialogroot.mainloop()

            dialog.destroy()

            print(self.strDialogResult)

            answer_text = '''
                            The university currently holds 11.966 students. Arranging housing is your own responsibility. International bachelors and masters students are also responsible for arranging their own housing. Within the program you have to travel between the two universities. Please bear in mind that the students have to cover the travel costs themselves (about 1200€ p. year).
                            Per year a TU/e student needs a budget of at least €10,000-€12,000 for rent, insurance, living expenses, books and local transportation. This is a minimum budget and is in addition to the tuition fee and any expenses incurred traveling from your home country to Eindhoven.
                            We are a high ranked top university in cooperation with regional, national and international industrial partners. 
                            There are very few scholarships open to international students wishing to follow a Bachelor's program. Check this link for more information:
                            EuFlex Technificent is the ideal matchmaker for TU/e students and industry. Our broad network of industry partners and TU/e students help both students and companies to find each other. Students can find vacancies for student Jobs and starter positions or learn more about a company during a company event. 
                            Yes you can do that. Email the academic advisor of your desired programme to find out more information about the procedure.
                            Yes, you can ask our students via email: askastudent@tue.nl
                            Per week, you will spend around 40 hours on studying.
                            '''
            input_ids = tokenizer.encode(self.strDialogResult, answer_text)

            # Report how long the input sequence is.
            #print('Query has {:,} tokens.\n'.format(len(input_ids)))

            # ======== Set Segment IDs ========
            # Search the input_ids for the first instance of the `[SEP]` token.
            sep_index = input_ids.index(tokenizer.sep_token_id)

            # The number of segment A tokens includes the [SEP] token istelf.
            num_seg_a = sep_index + 1

            # The remainder are segment B.
            num_seg_b = len(input_ids) - num_seg_a

            # Construct the list of 0s and 1s.
            segment_ids = [0]*num_seg_a + [1]*num_seg_b

            # There should be a segment_id for every input token.
            assert len(segment_ids) == len(input_ids)

            # ======== Evaluate ========
            # Run our example question through the model.
            start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                            token_type_ids=torch.tensor([segment_ids]), return_dict = False) # The segment IDs to differentiate question from answer_text

            # ======== Reconstruct Answer ========
            # Find the tokens with the highest `start` and `end` scores.
            answer_start = torch.argmax(start_scores)
            answer_end = torch.argmax(end_scores)

            # Get the string versions of the input tokens.
            tokens = tokenizer.convert_ids_to_tokens(input_ids)

            # Start with the first token.
            answer = tokens[answer_start]

            # Select the remaining answer tokens and join them with whitespace.
            for i in range(answer_start + 1, answer_end + 1):
                
                # If it's a subword token, then recombine it with the previous token.
                if tokens[i][0:2] == '##':
                    answer += tokens[i][2:]
                
                # Otherwise, add a space then the token.
                else:
                    answer += ' ' + tokens[i]
            print(answer)

            return self.strDialogResult

    def DialogResult(self, result):
        self.strDialogResult = result
        #This line quits from the MODAL STATE but doesn't close or destroy the modal dialog
        self.dialogroot.quit()

Example questions: 

How many students does the TU/e have?
How do you arrange housing? 
What is the cost of studying at the TU/e for internationals? 
Is the TU/e well ranked?
What is EuFlex?
How much time will you spend on studying?

In [9]:
if __name__ == '__main__':
    app = App()

How many students does the TU/e have?
11 . 966
How do you arrange housing?
your own responsibility
What is the cost of studying at the TU/e? 
€10 , 000 - €12 , 000
What is the cost of studying at the TU/e for internationals? 
€10 , 000 - €12 , 000
What is EuFlex?
the ideal matchmaker for tu / e students and industry
How much time will you spend on studying?
40 hours


KeyboardInterrupt: 